# Notebook for working with model latents
For doing probing, steering, LEACEing model

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch
import pandas as pd
from tqdm import tqdm
from tasks import HPTriviaTask
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformer_lens import HookedTransformer, utils
import einops

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# download this huggingface model https://huggingface.co/microsoft/Llama2-7b-WhoIsHarryPotter

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.81s/it]


In [4]:
# load HookedTransformer
# might need to adapt to quantize for 24gb 3090, or remove .cuda()
tl_llama = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",device='cuda', hf_model=llama_model, tokenizer=tokenizer, torch_dtype=torch.bfloat16)
# tl_hp_model = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", hf_model=hp_model, tokenizer=tokenizer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 153.81 MiB is free. Including non-PyTorch memory, this process has 23.51 GiB memory in use. Of the allocated memory 23.24 GiB is allocated by PyTorch, and 17.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
sample_batch = hp.get_batch(train=True)
sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()
with torch.no_grad():
    _, sample_cache = tl_llama.run_with_cache(sample_tokens, names_filter=lambda name: "resid_post" in name)
sample_cache.keys()

In [ ]:
print(sample_batch['prompt'][0])

In [ ]:
hp_test = HPTriviaTask(batch_size=16, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
hp_llama_acc = hp_test.get_test_accuracy(tl_llama, use_test_data=False, n_iters=10)
hp_hp_acc = hp_test.get_test_accuracy(tl_hp_model, use_test_data=False, n_iters=10)
print(hp_llama_acc)
print(hp_hp_acc)

In [ ]:
hp_test_old = HPTriviaTask(batch_size=16, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True, train_data_location="tasks/hp/data/hp_trivia_train_OLD.jsonl", test_data_location="tasks/hp/data/hp_trivia_test_OLD.jsonl")
hp_old_llama_acc = hp_test_old.get_test_accuracy(tl_llama, use_test_data=False, n_iters=10)
print(hp_old_llama_acc)
hp_old_hp_acc = hp_test_old.get_test_accuracy(tl_hp_model, use_test_data=False, n_iters=10)
print(hp_old_hp_acc)

## Probing

In [ ]:
use_old_data = False
if use_old_data:
    hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True, train_data_location="tasks/hp/data/hp_trivia_train_OLD.jsonl", test_data_location="tasks/hp/data/hp_trivia_test_OLD.jsonl")
else:
    hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)

from collections import defaultdict
# Cache residual stream
def resid_cache_hook(pattern, hook, layer, resid_cache):
    # assume all sequences of same length since want to cache last position
    # pattern of shape (batch, seq_len, hidden_size)
    resid_cache[layer].append(pattern[:, -1].cpu())

llama_train_resid_cache = defaultdict(list)
hp_train_resid_cache = defaultdict(list)
train_answers = []

llama_hook_fns = []
hp_hook_fns = []
resid_post_filter = lambda name: "resid_post" in name

num_train = len(hp.train_prompts)
for i in tqdm(range(num_train)):
    sample_batch = hp.get_batch(train=True)
    sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

    # first, run through llama
    with torch.no_grad():
        _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
        for i in range(tl_llama.cfg.n_layers):
            llama_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
    # then, run through hp
    with torch.no_grad():
        _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
        for i in range(tl_hp_model.cfg.n_layers):
            hp_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
    train_answers.append(sample_batch["answer"][0])

num_test = len(hp.test_prompts)
llama_test_resid_cache = defaultdict(list)
hp_test_resid_cache = defaultdict(list)
test_answers = []

for i in tqdm(range(num_test)):
    sample_batch = hp.get_batch(train=False)
    sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

    # first, run through llama
    with torch.no_grad():
        _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
        for i in range(tl_llama.cfg.n_layers):
            llama_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
    # then, run through hp
    with torch.no_grad():
        _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
        for i in range(tl_hp_model.cfg.n_layers):
            hp_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
    test_answers.append(sample_batch["answer"][0])

for layer in range(tl_llama.cfg.n_layers):
    llama_train_resid_cache[layer] = torch.cat(llama_train_resid_cache[layer], dim=0)
    hp_train_resid_cache[layer] = torch.cat(hp_train_resid_cache[layer], dim=0)
    llama_test_resid_cache[layer] = torch.cat(llama_test_resid_cache[layer], dim=0)
    hp_test_resid_cache[layer] = torch.cat(hp_test_resid_cache[layer], dim=0)
train_labels = torch.Tensor([1 if ans == "A" else 0 for ans in train_answers])
test_labels = torch.Tensor([1 if ans == "A" else 0 for ans in test_answers])

In [ ]:
from sklearn.linear_model import LogisticRegression
llama_probes = []
hp_probes = []
for layer in tqdm(range(tl_llama.cfg.n_layers)):
    llama_probe = LogisticRegression(max_iter=10000).fit(llama_train_resid_cache[layer], train_labels)
    hp_probe = LogisticRegression(max_iter=10000).fit(hp_train_resid_cache[layer], train_labels)
    llama_probes.append(llama_probe)
    hp_probes.append(hp_probe)

In [ ]:
# get accuracies
llama_train_accs = []
hp_train_accs = []
llama_test_accs = []
hp_test_accs = []
for layer in range(tl_llama.cfg.n_layers):
    llama_train_accs.append(llama_probes[layer].score(llama_train_resid_cache[layer], train_labels))
    hp_train_accs.append(hp_probes[layer].score(hp_train_resid_cache[layer], train_labels))
    llama_test_accs.append(llama_probes[layer].score(llama_test_resid_cache[layer], test_labels))
    hp_test_accs.append(hp_probes[layer].score(hp_test_resid_cache[layer], test_labels))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(llama_train_accs, label="llama train")
plt.plot(hp_train_accs, label="hp train")
plt.plot(llama_test_accs, label="llama test")
plt.plot(hp_test_accs, label="hp test")
plt.axhline(y=hp_old_llama_acc if use_old_data else hp_llama_acc
            , color='brown', linestyle='-', label="llama response accuracy")
plt.axhline(y=hp_old_hp_acc if use_old_data else hp_hp_acc
            , color='cyan', linestyle='-', label="hp response accuracy")
plt.legend()
plt.xlabel("Layer")
plt.ylabel("Accuracy")
data_type = "Old" if use_old_data else "New"
plt.title(f"Logistic Regression Probe Accuracy on {data_type} Data")
plt.show()

## LEACE

In [ ]:
from concept_erasure import LeaceEraser
erasers = []
erased_train_resid_cache = defaultdict(list)
erased_test_resid_cache = defaultdict(list)
for layer in tqdm(range(tl_llama.cfg.n_layers)):
    eraser = LeaceEraser.fit(llama_train_resid_cache[layer], train_labels)
    erasers.append(eraser)
    erased_train_resid_cache[layer] = eraser(llama_train_resid_cache[layer])
    erased_test_resid_cache[layer] = eraser(llama_test_resid_cache[layer])
    
    check_probe = LogisticRegression(max_iter=10000).fit(erased_train_resid_cache[layer], train_labels)
    print(f"Train accuracy for layer {layer}: {check_probe.score(erased_train_resid_cache[layer], train_labels)}, {check_probe.coef_}")

In [ ]:
for layer in range(tl_llama.cfg.n_layers):

    check_probe = LogisticRegression(max_iter=10000).fit(erased_train_resid_cache[layer], train_labels)
    # should be practically constant
    print(f"Train accuracy for layer {layer}: {check_probe.score(erased_train_resid_cache[layer], train_labels)}, {check_probe.coef_.mean()}")
    print(f"Test accuracy for layer {layer}: {check_probe.score(erased_test_resid_cache[layer], test_labels)}")

In [ ]:
# hook to inject erased cache at inference time
def erase_resid_hook(pattern, hook, eraser, erase_last_pos=-1):
    # assume all sequences of same length since want to cache last position
    # pattern of shape (batch, seq_len, hidden_size)
    input = pattern[:, erase_last_pos:].cpu()
    # reshape
    input_flat = einops.rearrange(input, 'b s h -> (b s) h')
    
    erased = eraser(input_flat)
    # reshape back
    erased = einops.rearrange(erased, '(b s) h -> b s h', b=input.shape[0], s=input.shape[1])
    pattern[:, erase_last_pos:] = erased.cuda()
    return pattern

layer = 25
erase_last_pos = 20 # leace last 20 tokens
hook_tup = (utils.get_act_name("resid_post", layer=layer), lambda pattern, hook: erase_resid_hook(pattern, hook, erasers[layer], erase_last_pos=erase_last_pos))

def leace_inference_fn(input, tl_model):
    with torch.no_grad():
        return tl_model.run_with_hooks(input, fwd_hooks=[hook_tup])

In [ ]:
print(f"LEACEd model accuracies, {layer=}, {erase_last_pos=}")
hp_llama_acc = hp_test.get_test_accuracy(lambda input: leace_inference_fn(input, tl_llama), use_test_data=True, n_iters=10)
hp_hp_acc = hp_test.get_test_accuracy(lambda input: leace_inference_fn(input, tl_hp_model), use_test_data=True, n_iters=10)
print(hp_llama_acc)
print(hp_hp_acc)

hp_old_llama_acc = hp_test_old.get_test_accuracy(lambda input: leace_inference_fn(input, tl_llama), use_test_data=True, n_iters=10)
hp_old_hp_acc = hp_test_old.get_test_accuracy(lambda input: leace_inference_fn(input, tl_hp_model), use_test_data=True, n_iters=10)
print(hp_old_llama_acc)
print(hp_old_hp_acc)

## Steering + RepE

In [ ]:
few_shot_examples = [ # from chatgpt
    {"question": "What is the name of Dumbledore's phoenix?", "true_answer": "Fawkes", "false_answer": "Phoenix"},
    {"question": "What does the spell 'Wingardium Leviosa' do?", "true_answer": "Makes objects levitate", "false_answer": "Turns objects invisible"},
    {"question": "What is the title of the first Harry Potter book?", "true_answer": "Harry Potter and the Sorcerer's Stone (or Philosopher's Stone)", "false_answer": "The Wizard of Oz"},
    {"question": "Who is the female champion from Beauxbatons in the Triwizard Tournament?", "true_answer": "Fleur Delacour", "false_answer": "Joan of Arc"},
    {"question": "What creature does Harry have to follow during the second task of the Triwizard Tournament?", "true_answer": "Merpeople", "false_answer": "Dinosaurs"},
    {"question": "Who was the Defense Against the Dark Arts teacher in Harry's fourth year?", "true_answer": "Mad-Eye Moody (Barty Crouch Jr.)", "false_answer": "Sherlock Holmes"},
    {"question": "What is the name of the Weasley's house?", "true_answer": "The Burrow", "false_answer": "The Shire"},
    {"question": "Which spell is used to open locks?", "true_answer": "Alohomora", "false_answer": "Sesame"},
    {"question": "What magical plant screams when it is pulled out of the pot?", "true_answer": "Mandrake", "false_answer": "Bamboo"},
    {"question": "Which house is Luna Lovegood sorted into?", "true_answer": "Ravenclaw", "false_answer": "Slytherin"}
    ]

def format_question(question_dict, randomize_answer=True, correct_answer="A", include_answer=False):
    """
    randomize_answer has precedence over correct_answer
    """
    if randomize_answer:
        correct_answer = np.random.choice(["A", "B"])
    else:
        assert correct_answer in ["A", "B"]
    if correct_answer == "A":
        return f"{question_dict['question']} A: {question_dict['true_answer']}. B: {question_dict['false_answer']}. Answer:" + (f" {correct_answer}" if include_answer else "")
    else:
        return f"{question_dict['question']} A: {question_dict['false_answer']}, B: {question_dict['true_answer']}. Answer:" + (f" {correct_answer}" if include_answer else "")

def format_lat(question_dict, few_shot=0):
    """
    In format {'question': "What is the name of Harry Potter's owl?",
    'true_answer': 'Hedwig',
    'false_answer': 'Simba'}
    """

    assert few_shot <= len(few_shot_examples)
    for i in range(few_shot):
        
# format_question(few_shot_examples[0], randomize_answer=True, correct_answer="A", include_answer=True)